In [ ]:
import os
import numpy as np
from skimage import io, transform,color
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [ ]:
# Set the paths to the directories containing PCOS and non-PCOS skin images
pcos_dir = 'pcos'
non_pcos_dir = 'nonpcos' 

In [ ]:
# Initialize empty lists to store image data and labels
images = []
labels = []

In [ ]:
# Load PCOS images and assign labels
for filename in os.listdir(pcos_dir):
    img = io.imread(os.path.join(pcos_dir, filename))
    img = img[:, :, :3]
    img = transform.resize(img, (100, 100,3))  # Resize the image to a desired size
    img_gray = color.rgb2gray(img)
    images.append(img_gray)
    labels.append(1)  # Assign label 1 for PCOS images

In [ ]:
# Load non-PCOS images and assign labels
for filename in os.listdir(non_pcos_dir):
    img = io.imread(os.path.join(non_pcos_dir, filename))
    img = img[:, :, :3]
    img = transform.resize(img, (100, 100,3))
    img_gray = color.rgb2gray(img)
    images.append(img_gray)
    labels.append(0)  # Assign label 0 for non-PCOS images

In [ ]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [ ]:
images = np.expand_dims(images, axis=-1)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.25, random_state=42)

In [ ]:
# Normalize the pixel values between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator object and specify the desired transformations
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.4
)

datagen.fit(X_train)

In [ ]:
# Convert labels to categorical format
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)))  # Update input shape
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu',  kernel_regularizer=regularizers.l2(0.03)))
model.add(Dropout(0.25))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
# Compile the model
import tensorflow as tf
LEARNING_RATE = 0.001 
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)